In [9]:
import os 
import pandas as pd
from collections import Counter
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import *
stemmer = PorterStemmer()

In [10]:
def stem(data):
    
    new = {}
    for k,v in data.items():
        new[k] = stemmer.stem(k)
    for old,new in new.items():
        data[new] = data.pop(old) 
        
    return data

In [11]:
def condProb(data,total_spam,total_ham,nof_spam,nof_ham):
    
    n = len(data)
    #print(n)
    for word in data.keys():
        #cond_spam  = (data[word][0]+1)/(total_spam-data[word][0] + n -1)
        #cond_ham   = (data[word][1]+1)/(total_ham-data[word][1] + n -1)
        cond_spam = (data[word][0]+1)/(total_spam+ n*1)
        cond_ham = (data[word][1]+1)/(total_ham + n*1)
        data[word] = data[word] + [cond_spam,cond_ham]
    #print(nof_spam)
    #print(nof_ham)
    prior_spam = nof_spam / (nof_spam+nof_ham)
    prior_ham = nof_ham / (nof_spam+nof_ham)
    #print(prior_spam)
    
    return data,prior_spam,prior_ham

In [12]:
def train():
    data = {}
    total_spam = 0
    total_ham = 0
    nof_spam = 0
    nof_ham = 0
    
    for filename in os.listdir(".\ham_train"):
        file = open('.\ham_train' + '\\' + filename, errors='ignore')
        wordcount = Counter(file.read().split())
        for item in wordcount.items():
            if item[0] in data:
                data[item[0]][1] += item[1]
            else:
                data[item[0]] = [0,item[1]]
            total_ham += item[1]
        nof_ham += 1

    for filename in os.listdir('.\spam_train'):
        file = open('.\spam_train'+'\\' +filename,errors='ignore')
        wordcount = Counter(file.read().split())
        for item in wordcount.items():
            if item[0] in data:
                 data[item[0]][0] += item[1]
            else:
                data[item[0]] = [item[1],0]
            total_spam += item[1]
        nof_spam += 1
    #print(nof_spam)
    data = stem(data)
    data_new,ps,ph = condProb(data,total_spam,total_ham,nof_spam,nof_ham)
    return data_new,ps,ph
    


In [72]:
def predict(text,d1,prior_spam,prior_ham):
    file = open(text,errors="ignore")
    wordcount = Counter(file.read().split())
    #print(wordcount.items())
    """
    l = []
    stop_words = set(stopwords.words('english'))
    for k in wordcount:
        #print(k)
        if k in stop_words:
            l.append(k)
    #print(l)
    for k in l:
        del wordcount[k]"""
        
    #print(wordcount)
    
    spam_score = np.log(prior_spam)
    ham_score = np.log(prior_ham)
    for item in wordcount.items():
        #print(item)
        text = item[0]
        
        count= item[1]
        #print(count)
        if text in d1:
            spam_score += np.log(d1[text][2])*count
            ham_score += np.log(d1[text][3])*count

    if spam_score>=ham_score:
        return "spam"
    else:
        return "ham"

In [69]:

def accuracy(d1,prior_spam,prior_ham):
    tp = 0
    tn = 0 
    fp = 0
    fn = 0
    path1 = r".\ham_train"
    for filename in os.listdir(path1):
        x = predict(path1 + "//" + filename,d1,prior_spam,prior_ham)
        if x== "spam":
            fn += 1
        if x == "ham":
            tp += 1
        
    path2 = r".\spam_train"
    for filename in os.listdir(path2):
        x = predict(path2 + "//" + filename,d1,prior_spam,prior_ham)
        if x == "spam":
            tn += 1
        if x == "ham":
            fp += 1

    accuracy = (tp+tn)/(tp+tn+fp+fn)
    f = open("results.txt","a")
    f.write("The accuracy is %.4f\n" %accuracy)
    f.close()
    
    return accuracy



In [70]:
def main():
    c=0
    d1,prior_spam,prior_ham = train()
    acc = accuracy(d1,prior_spam,prior_ham)
    
    

In [71]:
if __name__ == "__main__":
    main()